In [1]:
import scipy
import os
import galsim
import sys
import math
import cmath
import logging
import numpy as np
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import sympy
from sympy import Symbol, integrate 

In [2]:
# plotting 
def plotThreeGalaxies(image1, image2, image3, name1, name2, name3): 
    #create a plot 
    f, axs = plt.subplots(1, 3, figsize=(14,6))
    im0 = axs[0].imshow(image1.array, origin='lower', interpolation='None')
    axs[0].set_title(f'{name1}', fontsize=10)
    im1 = axs[1].imshow(image2.array, origin='lower', interpolation='None')    
    axs[1].set_title(f'{name2}', fontsize=10)
    im2 = axs[2].imshow(image3, origin='lower', interpolation='None')   
    axs[2].set_title(f'{name3}', fontsize=10)
    f.colorbar(im0)
    f.colorbar(im1)
    f.colorbar(im2)

## psf 

def makeSinglePSF():
    # want the FWHM to be 0.6" which is 3 pixels 
    psfobject = galsim.Gaussian(fwhm=0.6, flux=1.0)
    #visualize the PSF 
    psfimage = psfobject.drawImage(scale=0.2)
    return psfobject, psfimage 

def DetermineScaleFactors(psfimage): 
    # transform into fourier power function of the PSF 
    FFTpsf = galsim.fft.fft2(psfimage.array) 
    G = abs(FFTpsf)**2 # fourier power spectrum of the PSF 

    # Use G to determine rpp 
    # A is the area where the value is e^(-0.5) times the maximum of G (noiseless fourier power function of the psf) 
    gmax = G.max()
    ncriteria = 0 
    for pixel in G.flatten(): 
        if pixel > gmax*math.exp(-0.5):
            ncriteria += 1

    # rpp = scale radius of the PSF in fourier space 
    # converted number of pixels to an area of pixel (in arcsec^2) 
    rpp = (ncriteria*(0.2)*(0.2)/math.pi)**(1/2)

    #use this to calculate sigma for shapelet basis functions 
    beta = 0.85 # effective scale radius in fourier space, set to 0.85 (Li 2018)
    sigma = beta* rpp 

    alpha = 4 # effective scale in configuration space, set to 4 (Li 2018)

    return sigma, G 

# gradient creating function 

def MakeGradientGradualTopBottom(order, galaxysize):
    b = 2*order - 1/3 + 4/(405*order) + 46/(25515*order**2) +131/ (1148175*order**3) - 2194697/(30690717750*order**4) 
    FWHM = 0.2
    re = -FWHM*b/(math.log(0.5))
    newrows = [] 
    for x in range(18): 
        # return to flux 
        #specifiedflux = 1000+(x/18) 
        specifiedflux=10000
        sersicgalaxy = galsim.Sersic(n=order, half_light_radius= re, flux=specifiedflux)
        sersicgalaxyimage = sersicgalaxy.drawImage(nx=18, ny=18, scale=0.2)
        if x == 0: 
            gradientarray = sersicgalaxyimage.array[x]
        else: 
            newrows.append(sersicgalaxyimage.array[x])
    
    for array in newrows: 
        gradientarray = np.vstack([gradientarray, array])
    
    #convert gradientArray to image 
    gradientimage = galsim.Image(gradientarray, copy=True)

    #convert the fits file to an object 
    gradientobject = galsim.InterpolatedImage(image=gradientimage, scale=0.2)
    
    return gradientobject, gradientimage

# convolution, treatment 
def ConvolveGalaxyPSF(galaxy, psf):
    # convolve the galaxy and psf (psf, sersic_gal) (integral 2d of psf*galaxy) 
    colv = galsim.Convolution(galaxy, psf, real_space=True)
    # take the fourier transform, need to make this into an image 
    convolveimage = colv.drawImage(nx=18, ny=18, scale=0.2)
    FFTcolv = galsim.fft.fft2(convolveimage.array) 
    F = abs(FFTcolv)**2 
    return F 

def RemovePSF(F, G):
    Fnopsf = F/ G
    #plt.imshow(Fnopsf, origin='lower', interpolation='None')
    #plt.title("Fourier power function, psf removed")
    #plt.colorbar()
    return Fnopsf

def X00Star(r, sigma): #evaluate X00star for a specified value of r     
    b = math.exp(-(r**2)/2*sigma**2)
    c = (math.pi**(1/2)*sigma**3)
    return b/c

def X22StarTrig(r, t, sigma): # using Eulers formula
    a = r**2
    b = math.exp(-(r**2)/2*sigma**2) 
    z = complex(math.cos(2*t), math.sin(2*t)) 
    d = math.pi**(1/2)*2**(1/2)*sigma**3
    value = a*b*z/d
    return value

def calculateListMnm(Fnopsf, sigma):
    M00array = []
    M22arrayreal = []
    M22arrayimag = []
    for idx, val in np.ndenumerate(Fnopsf): 
        # determine the position in the array, convert to its position in F.S. (rho, phi)

        nr = idx[0] 
        nc = idx[1] 
        x = nc - 9 # index of the column 
        y = 9 - nr # index of the row 
        r = (x**2 + y**2)**(1/2) # rho 
        
        if x == 0: 
            if y > 0: 
                t = math.pi/ 2
            elif y < 0: 
                t = 3*math.pi/2 
        else:     
            t = np.arctan(y/x) 

        # evaluate the shapelet basis functions at position 
        # evaluate function X00star 
        X00= X00Star(r, sigma)         
        Xval00 = X00 * val
        M00array.append(Xval00)

        # evaluate function X22star 
        X22 = X22StarTrig(r, t, sigma)
        X22real = X22.real
        X22imag = X22.imag
        
        M22arrayreal.append(X22real * val) 
        M22arrayimag.append(X22imag * val) 
    
    return M00array, M22arrayreal, M22arrayimag

def determineEllipticities(M00array, M22arrayreal, M22arrayimag):
    M00 = sum(M00array)
    M22C = sum(M22arrayreal)
    M22S = sum(M22arrayimag)
    C = 1 # adjusts the relative weight between galaxies of different luminosities
    e1 = M22C / (M00 + C) 
    e2 = M22S / (M00 + C) 
    return e1, e2 

In [6]:
def simulatedGalaxyWithConversion(order, galaxysize, flux, plot, printe1e2, convert): 
    # generate a PSF 
    psfobject, psfimage = makeSinglePSF() 
    sigma, G = DetermineScaleFactors(psfimage)

    # generate a galaxy 
    b = 2*order - 1/3 + 4/(405*order) + 46/(25515*order**2) +131/ (1148175*order**3) - 2194697/(30690717750*order**4) 
    FWHM = 0.2
    re = -FWHM*b/(math.log(0.5))
    
    galaxyobject = galsim.Sersic(n=order, half_light_radius= re, flux=flux)
    galaxyimage = galaxyobject.drawImage(scale=0.2, nx=18, ny=18)

    if convert == True: 
        #convert galaxyimagearray to image 
        galaxyimage = galsim.Image(galaxyimage.array, copy=True)
        # from iterations, I know the issue is not here in the image portion 

        #convert the fits file to an object 
        galaxyobject = galsim.InterpolatedImage(image=galaxyimage, scale=0.2, flux=10000)
        
    # transform into FS
    F = ConvolveGalaxyPSF(galaxyobject, psfobject)
    Fnopsf = RemovePSF(F, G)

    M00array, M22arrayreal, M22arrayimag = calculateListMnm(Fnopsf, sigma)
    e1, e2 = determineEllipticities(M00array, M22arrayreal, M22arrayimag)

    if plot == True: 
        plotThreeGalaxies(psfimage, galaxyobject, F, "psf", "galaxy", "convolved, F")

    if printe1e2 == True:
        print("e1, e2:", e1, e2) 
    
    return e1, e2

In [7]:
a,b = simulatedGalaxyWithConversion(order=1, galaxysize=16*16, flux=10000, plot=False, printe1e2=True, convert=False)

e1, e2: 1.0228934600114813e-06 -0.0032300583054547124


In [8]:
a,b = simulatedGalaxyWithConversion(order=1, galaxysize=16*16, flux=10000, plot=False, printe1e2=True, convert=True)

e1, e2: 1.7058811419701232 -0.00010649666155869542
